In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from utils import *
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Helvetica'

description =(
'''From the previous experiments, there are different model shapes derived.
Lets take each of these shapes and do IMP with 20 pruning levels.
'''
)

In [ ]:
sweep_ids = ['p30nbq46', 'r0b16unk', 'ocfuu4ip']
try: print('Histories already loaded.') if hc else None
except: hc = load_histories_and_configs(sweep_ids)

In [ ]:
_split = make_split_df(hc)
_shape = make_df(hc, 'model_shape').map(lambda shape: shape[1])
_pr = make_df(hc, 'pruning_rate')
_splitrange = make_splitrange_df(hc)
_first, _last = make_dfs_at_splitrange(hc, _splitrange)

# average over the seeds via pruning_levels (sweep parameters)
ydf = pd.concat([_split, _shape], axis=1).groupby('model_shape').sum()
xdf = pd.concat([_pr, _shape], axis=1).groupby('model_shape').mean()

In [ ]:
title='Network Splitting \nseveral architectures, 20 pruning levels, adaptive pruning rate'

fig, axes = plt.subplots(nrows=2, figsize=(16,9), sharey=True, sharex=False)
fig.suptitle(title, fontweight='regular', fontsize=24, fontname='Helvetica')

ydf[splitters.keys()].plot.bar(
    ax=axes[0],
    width=0.95,  
    color=splitters.values(),
    rot=0,
    stacked=True
)
ydf[nonsplitters.keys()].plot.bar(
    ax=axes[1],
    width=0.95,  
    color=nonsplitters.values(),
    rot=0,
    stacked=True
)

# x axis data
xticks = np.arange(len(xdf.index))
xticklabels = [f'{l}\n{pr*100:.1f}%' for l, pr in xdf['pruning_rate'].items()]

for ax in axes:
    ax.set_yticks(np.arange(np.max(ydf)+1))
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels, fontsize=10, color='gray')
    ax.set_ylabel('num models', fontsize=14)
    ax.set_xlabel('hidden-dim\np-rate', fontsize=10, color='gray')
    ax.xaxis.set_label_coords(0.0, -0.024)

plt.tight_layout(
    #rect=[0, 0.03, 1, 0.95]
)
set_style(axes)
plt.savefig("test.svg")

In [ ]:
group_df = _shape
source_df = _first
source_keys = [
    'active-weights-abs', 
    'zombious-abs',
    ]

group = make_group(group_df, source_df, source_keys)

for key in source_keys:
    df = mean_std_from_group(group, key)
    errorbar_from_df(df, label=key, marker='o', alpha=0.6)
plt.xscale('log')
plt.title('Metrics at split iteration - 20 pruning levels ')
plt.xlabel('number of hidden neurons')

In [ ]:
group_df = _shape
source_df = _first
source_df = _last
source_keys = [
    #'active-features-rel',  
    'active-weights-rel',    
    'accuracy'
]

group = make_group(group_df, source_df, source_keys)

plt.figure(figsize=(12, 4)) 
for key in source_keys:
    df = mean_std_from_group(group, key).reset_index(drop=True)
    plt.errorbar(
        x=df.index, 
        y=df['y'], 
        yerr=df['yerr'],
        marker='o',
        alpha=0.7,
        label=key,
    )

plt.title('Metrics of split networks (mean, std of 4 runs)')
plt.xticks(ticks=np.arange(len(df['x'])), labels=df['x'])
plt.xlabel('number of hidden neurons')
plt.legend()

In [ ]:
group_df = _shape
source_df = _last
source_df = _first
source_keys = [
    #'pparams',    
    'active-weights-abs'
]
source_labels = ['available parameters', 'active parameters']

group = make_group(group_df, source_df, source_keys)

plt.figure(figsize=(12, 4)) 
for key, label in zip(source_keys, source_labels):
    df = mean_std_from_group(group, key).reset_index(drop=True)
    plt.errorbar(
        x=df.index, 
        y=df['y'], 
        yerr=df['yerr'],
        marker='o',
        alpha=0.7,
        label=label,
    )

plt.title('Active weights vs Available weights at first split\n(mean, std of 4 runs)')
plt.xticks(ticks=np.arange(len(df['x'])), labels=df['x'])
plt.xlabel('number of hidden neurons')
plt.legend()
plt.savefig("test.svg")

In [ ]:
group_df = _shape
source_df = _last
source_df = _first

plt.figure(figsize=(12, 4)) 

group = make_group(group_df, _first, ['active-weights-abs'])
df = mean_std_from_group(group, 'active-weights-abs').reset_index(drop=True)
plt.errorbar(
    x=df.index, 
    y=df['y'], 
    yerr=df['yerr'],
    marker='o',
    alpha=0.7,
    label='active weights (first iter)',
)

group = make_group(group_df, _last, ['active-weights-abs'])
df = mean_std_from_group(group, 'active-weights-abs').reset_index(drop=True)
plt.errorbar(
    x=df.index, 
    y=df['y'], 
    yerr=df['yerr'],
    marker='o',
    alpha=0.7,
    label='active weights (last iter)',
)

plt.title('Networks at first and last split iteration\n(mean, std of 4 runs)')
plt.xticks(ticks=np.arange(len(df['x'])), labels=df['x'])
plt.xlabel('number of hidden neurons')
plt.legend()
plt.savefig("test.svg")